In [13]:
import cv2
import pydicom as dicom
import keras
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import os
import pandas as pd
from PIL import Image
import shutil
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline

In [3]:
#   Load datasets
calc_train_df_original = pd.read_csv("Calc_Train/CBIS-DDSM/calc_case_description_train_set.csv")
calc_test_df_original = pd.read_csv("Calc_Test/CBIS-DDSM/calc_case_description_test_set.csv")

mass_train_df_original = pd.read_csv("Mass_Train/CBIS-DDSM/mass_case_description_train_set.csv")
mass_test_df_original = pd.read_csv("Mass_Test/CBIS-DDSM/mass_case_description_test_set.csv")

#   Remove unwanted labels. In our case the last three columns

calc_train_df_original.head()

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.959...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...,Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9...
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,Calc-Training_P_00005_RIGHT_MLO/1.3.6.1.4.1.95...,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....,Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1....
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.95...
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,Calc-Training_P_00007_LEFT_MLO/1.3.6.1.4.1.959...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...,Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9...
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN_WITHOUT_CALLBACK,3,Calc-Training_P_00008_LEFT_CC/1.3.6.1.4.1.9590...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...,Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.95...


In [6]:
#   Preprocess data
##  Change values of all benign without callback to benign

def createDF(patient_id, pathology):
    data = {
        "patient_id": patient_id,
        "pathology": pathology
    }
    return pd.concat( data, axis=1 )

calc_train_data = createDF(calc_train_df_original["patient_id"], calc_train_df_original["pathology"]) 
calc_test_data = createDF(calc_test_df_original["patient_id"], calc_test_df_original["pathology"])
mass_train_data = createDF(mass_train_df_original["patient_id"], mass_train_df_original["pathology"])
mass_test_data = createDF(mass_test_df_original["patient_id"], mass_test_df_original["pathology"])

,patient_id,pathology
0,P_00005,MALIGNANT
1,P_00005,MALIGNANT
2,P_00007,BENIGN
3,P_00007,BENIGN
4,P_00008,BENIGN_WITHOUT_CALLBACK
...,...,...
1541,P_02566,MALIGNANT
1542,P_02572,MALIGNANT
1543,P_02572,MALIGNANT
1544,P_02584,BENIGN


In [14]:
#   Data preprocessing
##  Steps
### 1. Extract png from dcm file
### 2. Rename file with directory file name

def extractDCMAndSave( dcmFilePath, directory, filename ):
    ds = dicom.dcmread( dcmFilePath )
    ds = ds.pixel_array * 128
    img = Image.fromarray(ds, 'RGB')
    img.save(directory + filename + ".png" )

In [23]:
#   Calc Test
for i in range(len(calc_test_df_original)):
    filename = calc_test_df_original.loc[i, "patient_id"] + "_" + calc_test_df_original.loc[i, "left or right breast"] + "_" + calc_test_df_original.loc[i, "image view"]
    directory = "calcifications_test/"
    path = calc_test_df_original.loc[i, "cropped image file path"]
    pathList = path.split('\n')
    extractDCMAndSave( "Calc_Test/" + str(pathList[0]), directory, filename )

FileNotFoundError: [Errno 2] No such file or directory: 'Calc_Test/Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.100.1.2.161465562211359959230647609981488894942/1.3.6.1.4.1.9590.100.1.2.419081637812053404913157930753972718515/000001.dcm'